# RD sim on spheres/metaballs

In [ ]:
%load_ext autoreload
%autoreload 1

import os
import itertools
import colorsys

import numpy as np
from joblib import Parallel, delayed
from PIL import Image
from vapory import *

from rdmb import *
%aimport rdmb

%matplotlib inline

### sim_set
|     |
|-----|
| p0  |
| p1  | 
| h12 |
| p2  |
| p3  |
#### p: parents, h: hybrids

In [ ]:
def sim_parent(file_base, out_dir, A, C, seed=42):
    
    np.random.seed(seed)
    mb_obj_file = "mbobj/mb_1.obj.gz"
    
    fb_p = out_dir + "/" + file_base + "_A{:.4f}_C{:.4f}_seed-{:d}".format(A, C, seed)
    rdmb.rdmb_uni_AC(mb_obj_file, fb_p, A=A, C=C, max_ite=2000)
    
    return fb_p

In [ ]:
def sim_hybrid(file_base, out_dir, As, Cs, form=12, seed=42):
    
    np.random.seed(seed)
    mb_obj_file = "mbobj/mb_3b.obj.gz"

    fb_h = out_dir + "/" + file_base + "_h12_seed-{:d}".format(seed)
    rdmb.rdmb_grad_AC(mb_obj_file, fb_h, -1.6, 1.4,
                          As[2], As[1], Cs[2], Cs[1], max_ite=2000)

    return fb_h

In [ ]:
AC_sets = [
    [[0.08, 0.08, 0.08, 0.08], [0.01, 0.06, 0.27, 0.307]],
    [[0.07, 0.075, 0.116, 0.12], [0.08, 0.08, 0.08, 0.08]],
    [[0.075, 0.08, 0.08, 0.12], [0.08, 0.01, 0.27, 0.08]],  
]

In [ ]:
As = AC_sets[0][0]
Cs = AC_sets[0][1]

display(As, Cs)

### h12

In [ ]:
%%time

np.random.seed(42)

out_dir = "rd_out/set2"

try:
    os.makedirs("./"+out_dir)
except FileExistsError:
    pass

fb_hs = Parallel(n_jobs=1, verbose=1, max_nbytes=None)(
            [delayed(sim_hybrid)(file_base="hs",
                                 out_dir=out_dir,
                                 As=As, Cs=Cs,
                                 form=i, seed=np.random.randint(10000))
             for i in [12]])

In [ ]:
pngfname = rdmb.rdmb_povray(fb_hs[0], 2000, width=600, height=400, angle=15)
display(Image.open(pngfname))


### p0-p3

In [ ]:
%%time

np.random.seed(42)

out_dir = "rd_out/set2"

try:
    os.makedirs("./"+out_dir)
except FileExistsError:
    pass

fb_ps = Parallel(n_jobs=4, verbose=1, max_nbytes=None)(
            [delayed(sim_parent)(file_base="ps",
                                 out_dir=out_dir,
                                 A=AC[0], C=AC[1],
                                 seed=np.random.randint(10000))
             for AC in zip(As, Cs)])

In [ ]:
fb_ps = ['rd_out/set2/ps_A0.0800_C0.0100_seed-7270',
         'rd_out/set2/ps_A0.0800_C0.0600_seed-860',
         'rd_out/set2/ps_A0.0800_C0.2700_seed-5390',
         'rd_out/set2/ps_A0.0800_C0.3070_seed-5191']

In [ ]:
fb_hs = ['rd_out/set2/hs_h12_seed-7270']

In [ ]:
def sim_set_scene(fb, rotx=0, roty=0, rotz=0, mode="C"):
    
    vs, ucs, A, C = rdmb.load_rd_mb(fb)
        
    camera = Camera('location', [0, 0, -45],
                    'look_at', [0, 0, 0],
                    'angle', 6,
                    'right x*image_width/image_height')
    
    light = LightSource([-10, 10, -20],
                        'color', [1.0, 1.0, 1.0], 'parallel')
    light2 = LightSource([10, -6, -20],
                         'color', [0.2, 0.2, 0.2], 'parallel')
    background = Background('color', [1, 1, 1, 1])

    spheres = []
    xd = 3.0
    # xdd = 0.1
    xdd = 0.2

    spheres += rdmb.sph(vs, ucs,
                        np.ones(len(vs))*A,
                        np.ones(len(vs))*C,
                        0, 0, 0, rotx=rotx, roty=roty, rotz=rotz, mode=mode)
    
    objects = [light, light2, background] + spheres

    scene = Scene(camera, objects=objects)
    
    return scene

In [ ]:
scenes = [sim_set_scene(fb=fb_ps[0],
                        rotx=0,
                        rotz=0,
                        mode="C")]

png_file = "rdmb_p0_0.png"
scenes[0].render(png_file,
                 width=800,
                 height=800,
                 output_alpha=True,
                 antialiasing=0.001)

!convert $png_file -gravity center -crop 400x400+0+0 $png_file
display(Image.open(png_file))

In [ ]:
scenes = [sim_set_scene(fb=fb_ps[1],
                        rotx=0,
                        rotz=0,
                        mode="C")]

png_file = "rdmb_p1_0.png"
scenes[0].render(png_file,
                 width=800,
                 height=800,
                 output_alpha=True,
                 antialiasing=0.001)

!convert $png_file -gravity center -crop 400x400+0+0 $png_file
display(Image.open(png_file))

In [ ]:
scenes = [sim_set_scene(fb=fb_hs[0],
                        rotx=195,
                        rotz=90,
                        mode="C")]

png_file = "rdmb_h12_0.png"
scenes[0].render(png_file,
                 width=800,
                 height=800,
                 output_alpha=True,
                 antialiasing=0.001)

!convert $png_file -gravity center -crop 600x800+0+0 $png_file
display(Image.open(png_file))

In [ ]:
scenes = [sim_set_scene(fb=fb_ps[2],
                        rotx=0,
                        rotz=0,
                        mode="C")]

png_file = "rdmb_p2_0.png"
scenes[0].render(png_file,
                 width=800,
                 height=800,
                 output_alpha=True,
                 antialiasing=0.001)

!convert $png_file -gravity center -crop 400x400+0+0 $png_file
display(Image.open(png_file))

In [ ]:
scenes = [sim_set_scene(fb=fb_ps[3],
                        rotx=0,
                        rotz=0,
                        mode="C")]

png_file = "rdmb_p3_0.png"
scenes[0].render(png_file,
                 width=800,
                 height=800,
                 output_alpha=True,
                 antialiasing=0.001)

!convert $png_file -gravity center -crop 400x400+0+0 $png_file
display(Image.open(png_file))